In [2]:
import market_data
from datetime import datetime

In [5]:
# Binance API parameters
traded_assets = ['BTC', 'ETH', 'ADA', 'DOGE']
start_date = "Jan 01, 2021"
end_date = "Dec 31, 2021"
stable_assets = ['USDT']
time_horizons = ['1hr', '12hr', '1wk']

# EMA parameters
alpha_values = [0.25, 0.5, 0.75]

In [6]:
"""
Fetch data with all parameter combinations
"""
for traded_asset in traded_assets:
    for stable_asset in stable_assets:
        for time_horizon in time_horizons:
            for alpha in alpha_values:
                df_raw = market_data.fetchBinance(
                        traded_asset=traded_asset,
                        stable_asset=stable_asset,
                        time_frame=time_horizon,
                        start_date=start_date,
                        end_date=end_date
                    )
                
                # only 'open' and 'close' needed for moving average calculation
                df = df_raw[['open', 'close', 'closeTime']].copy()
                df['close_ema'] = df['close'].ewm(alpha=alpha).mean()

                # convert time from unix to human readable for r plotting
                for i in range(len(df)):
                    df.loc[i, 'close_time_parsed'] = datetime.utcfromtimestamp(df.loc[i, 'closeTime']//1_000).strftime('%Y-%m-%d %H:%M:%S')

                df.to_csv(f'data/{traded_asset}_{stable_asset}_{time_horizon}_{alpha}_ema.csv', index=False)
